# Experimenting With Solar Libraries

what should we do?
- experiment with pvlib
- pull in NREL PySAM
- pull real irradiance data from NSRDB
- experiment with different array configurations in Vancouver and Jakarta

In [ ]:
import pvlib

import pandas as pd

import matplotlib.pyplot as plt

In [ ]:
coordinates = [
    (32.2, -111.0, 'Tucson', 700, 'Etc/GMT+7'),
    (35.1, -106.6, 'Albuquerque', 1500, 'Etc/GMT+7'),
    (37.8, -122.4, 'San Francisco', 10, 'Etc/GMT+8'),
    (52.5, 13.4, 'Berlin', 34, 'Etc/GMT-1'),
]

sandia_modules = pvlib.pvsystem.retrieve_sam('SandiaMod')

sapm_inverters = pvlib.pvsystem.retrieve_sam('cecinverter')

module = sandia_modules['Canadian_Solar_CS5P_220M___2009_']

inverter = sapm_inverters['ABB__MICRO_0_25_I_OUTD_US_208__208V_']

temperature_model_parameters = pvlib.temperature.TEMPERATURE_MODEL_PARAMETERS['sapm']['open_rack_glass_glass']

In [ ]:
tmys = []

for location in coordinates:
    latitude, longitude, name, altitude, timezone = location
    weather = pvlib.iotools.get_pvgis_tmy(latitude, longitude)[0]
    weather.index.name = "utc_time"
    tmys.append(weather)

In [ ]:
system = {'module': module, 'inverter': inverter,
          'surface_azimuth': 180}


energies = {}

for location, weather in zip(coordinates, tmys):
    latitude, longitude, name, altitude, timezone = location
    system['surface_tilt'] = latitude

    # get a dataframe of the solar position in the sky.
    solpos = pvlib.solarposition.get_solarposition(
        time=weather.index,
        latitude=latitude,
        longitude=longitude,
        altitude=altitude,
        temperature=weather["temp_air"],
        pressure=pvlib.atmosphere.alt2pres(altitude),
    )

    # get the extraterrestial radiation, the radiation in the absence of an atmosphere
    dni_extra = pvlib.irradiance.get_extra_radiation(weather.index)

    # get the relative airmass at sea level, without airpressure
    airmass = pvlib.atmosphere.get_relative_airmass(solpos['apparent_zenith'])

    # get the atmospheric pressure at altitude
    pressure = pvlib.atmosphere.alt2pres(altitude)

    # get the absolute airmass, which is pressure adjusted.
    am_abs = pvlib.atmosphere.get_absolute_airmass(airmass, pressure)

    # calculate the angle of incidence of the solar vector on a surface
    aoi = pvlib.irradiance.aoi(
        system['surface_tilt'],
        system['surface_azimuth'],
        solpos["apparent_zenith"],
        solpos["azimuth"],
    )

    # extraterrestial irradiation gets converted into a dataframe of POA irradiance
    total_irradiance = pvlib.irradiance.get_total_irradiance(
        system['surface_tilt'],
        system['surface_azimuth'],
        solpos['apparent_zenith'],
        solpos['azimuth'],
        weather['dni'],
        weather['ghi'],
        weather['dhi'],
        dni_extra=dni_extra,
        model='haydavies',
    )

    # get cell temperature
    cell_temperature = pvlib.temperature.sapm_cell(
        total_irradiance['poa_global'],
        weather["temp_air"],
        weather["wind_speed"],
        **temperature_model_parameters,
    )

    # get the irradiance absorbed by a cell
    effective_irradiance = pvlib.pvsystem.sapm_effective_irradiance(
        total_irradiance['poa_direct'],
        total_irradiance['poa_diffuse'],
        am_abs,
        aoi,
        module,
    )

    # get the dc characteristics of the module array
    dc = pvlib.pvsystem.sapm(effective_irradiance, cell_temperature, module)

    # convert dc power and dc voltage to ac power
    ac = pvlib.inverter.sandia(dc['v_mp'], dc['p_mp'], inverter)

    # sum annual energy
    annual_energy = ac.sum()
    energies[name] = annual_energy


energies = pd.Series(energies)

print(energies)

energies.plot(kind='bar', rot=0)


plt.ylabel('Yearly energy yield (W hr)')

In [ ]:
from pvlib.pvsystem import PVSystem, Array, FixedMount

from pvlib.location import Location

from pvlib.modelchain import ModelChain

In [ ]:
energies = {}

for location, weather in zip(coordinates, tmys):
    latitude, longitude, name, altitude, timezone = location
    location = Location(
        latitude,
        longitude,
        name=name,
        altitude=altitude,
        tz=timezone,
    )
    mount = FixedMount(surface_tilt=latitude, surface_azimuth=180)
    array = Array(
        mount=mount,
        module_parameters=module,
        temperature_model_parameters=temperature_model_parameters,
    )
    system = PVSystem(arrays=[array], inverter_parameters=inverter)
    mc = ModelChain(system, location)
    mc.run_model(weather)
    annual_energy = mc.results.ac.sum()
    energies[name] = annual_energy


energies = pd.Series(energies)


print(energies)






energies.plot(kind='bar', rot=0)


plt.ylabel('Yearly energy yield (W hr)')

In [ ]:
df = pvlib.pvsystem.retrieve_sam('cecmod')

In [ ]:
df.T